In [ ]:
import wandb
wandb.login()

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import shutil
import os
import random
import pytorch_lightning as pl
from types import SimpleNamespace
import random

%matplotlib inline


data_prefix='/kaggle/input/inaturalist12k/Data/inaturalist_12K/'

# resize and normalizing images
transform = transforms.Compose([
                        transforms.Resize((224, 224)),
                        transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
                        ])

# Since we now know the best hyperparamters,we will use the full dataset to train the model.
train_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'val', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

idx_to_class = {v:k for k, v in train_dataset.class_to_idx.items()}


In [ ]:
def getActivation(function): # activations
    if function=='ReLU':
        return nn.ReLU()
    if function=='GELU':
        return nn.GELU()
    if function=='SiLU':
        return nn.SELU()
    if function=='Mish':
        return nn.Mish()
    return nn.ReLU() # if no match

def soft_max(x):
    return np.exp(x)/np.sum(np.exp(x),axis=0) # softmax for nomalziing probabilties
    
    

class Model(pl.LightningModule): #Building mode
    def __init__(self,config):
        
        super().__init__()
        self.learning_rate=config.learning_rate
        layers=[]
        input_channels=3
        num_layers=5
        kernel_size=3
        kernel_stride=1
        max_pool_size=2
        max_pool_stride=2
        filters=[]
        if(config.filter_organization=='same'):
            filters=[config.filter_size]*num_layers
        elif(config.filter_organization=='double'):
            filters.append(config.filter_size)
            for i in range(4):
                filters.append(filters[-1]*2)
        elif(config.filter_organization=='halve'):
            filters.append(config.filter_size)
            for i in range(4):
                filters.append(filters[-1]//2)

        filters.insert(0,input_channels)
        out_height=224
        for i in range(5):
            layers.append(nn.Conv2d(filters[i],filters[i+1],kernel_size = kernel_size))
            out_height=(out_height-kernel_size)//kernel_stride+1
            layers.append(nn.MaxPool2d(kernel_size = max_pool_size,stride = max_pool_stride))  
            out_height=out_height//max_pool_stride
            layers.append(getActivation(config.activation))
            if(config.batch_normalisation=='Yes'):
                layers.append(nn.BatchNorm2d(filters[i+1]))
        layers.append(nn.Flatten())
        layers.append(nn.Dropout(config.dropout))
        layers.append(nn.Linear(out_height*out_height*filters[-1],config.dense_layer_size))
        layers.append(nn.Linear(config.dense_layer_size,10))
        self.net = nn.Sequential(*layers)
        self.loss = nn.CrossEntropyLoss()
        self.train_loss=[]
        self.train_acc=[]
        
        
    def forward(self,x):
        return self.net(x)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr= self.learning_rate)

    def training_step(self,batch,batch_idx):
        X,Y = batch
        output = self(X)
        loss = self.loss(output,Y)
        acc = (output.argmax(dim = 1) == Y).float().mean()
        self.train_loss.append(loss)
        self.train_acc.append(acc)
        return loss
    def predict_step(self, batch, batch_idx):
        X, Y = batch
        preds = self.net(X)
        return preds
    
   

    def on_train_epoch_end(self):
        train_loss=sum(self.train_loss)/len(self.train_loss)
        train_acc=sum(self.train_acc)/len(self.train_acc)
        self.train_acc=[]
        self.train_loss=[]
        print(f"Epoch: {self.current_epoch} train accuracy :{train_acc:.2f} train_loss :{train_loss:.2f}")
        wandb.log({'train_acc':train_acc,'train_loss':train_loss})

In [ ]:
# best configuration found after hyperparameter search
best_config={
    'filter_size':32,
    'data_augumentation':"Yes",
    'batch_normalisation':"Yes",
    'dropout':0,
    'filter_organization':'double',
    'activation':'Mish',
    'epochs':10,
    'dense_layer_size':256,
    'learning_rate':0.0001,
}

In [ ]:
def main(params):
    if(params.data_augumentation=='Yes'): # loading train data based on data augumentation is enabled or not.
        transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),
                   ])
    else:
        transform = transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
                    ])


    train_dataset = torchvision.datasets.ImageFolder(root=data_prefix+'train', transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    model = Model(params) 
    trainer = pl.Trainer(max_epochs=params.epochs,devices=1,accelerator="gpu") 
    trainer.fit(model,train_loader) #fit the model
    return model,trainer


In [ ]:
def calc_acc(data_loader,targets): # to calculate accuracy of the model.
  preds = trainer.predict(model, data_loader)
  preds = torch.concat(preds)
  preds = preds.argmax(axis=1) #computing predicted classes.
  preds=preds.numpy()
  targets=np.array(targets)
  return np.sum(preds==targets)/len(targets)

In [ ]:
if __name__=='__main__':
    wandb.init(project='Best Model',config=best_config)
    params=dict(best_config)
    params=SimpleNamespace(**params)
    run_name=f'FZ-{params.filter_size} AF - {params.activation} filter_org- {params.filter_organization} batch_norm -{params.batch_normalisation} data_aug -{params.data_augumentation} dropout- {params.dropout}'
    wandb.run.name=run_name
    model,trainer=main(params)
    print("training completed")
    test_accuracy=calc_acc(test_loader,test_dataset.targets)
    print("*"*50)
    print(f"Final Test accuracy {test_accuracy:.2f}")
    print("*"*50)
    wandb.log({'test accuracy':test_accuracy}) #logging and test accuracy

In [ ]:
# to print sample images and their prediction in 

probs = trainer.predict(model, test_loader)
probs = torch.concat(probs)
probs=probs.numpy()
probs=soft_max(probs.T).T

preds=probs.argmax(axis=1)
targets=np.array(test_dataset.targets)


fig, ax = plt.subplots(10, 3, figsize = (10, 20),constrained_layout = True)
for i in range(10):
  random_idx=random.sample(set(np.argwhere(targets==i).flatten()),3)
  count=0
  for j in range(3):
      x=test_dataset[random_idx[count]][0]
      np.clip(x,0,1)
      y=test_dataset[random_idx[count]][1]
      x=x.permute(1,2,0).numpy()
      x=np.clip(x,0,1)
      ax[i][j].imshow(x)
      pred_class=preds[random_idx[count]]
      pred_prob=probs[random_idx[count]][pred_class]
      s=f'True : {idx_to_class[y]} \nPred : {idx_to_class[pred_class]} ({pred_prob:.2f})'
      if(y==pred_class): 
          ax[i][j].set_title(s,color='g')
      else:
        ax[i][j].set_title(s,color='r')
        
      count+=1
fig.tight_layout(pad=2.0)
wandb.log({'test data prediction':wandb.Image(plt)})
plt.title("Sample Images prediction")
plt.savefig('fig.jpg')
plt.show()